# Lab note assistant

This notebook demonstrates a lab note assistant: Video-to-lab-note conversion using Vertex AI

Converting Videos-to-lab-note involves three steps: 
1. Protocol finder: Select protocol which best captures the step being performed in the video
2. Video comparing to ground-of-truth protocol → lab notes + errors in procedure
3. Analytics based on benchmark dataset: automatic comparison of errors found by lab note assistent vs actual errors

In this notebook, I will focus on the step two and three - Compare video with protocol.

In [1]:
from __future__ import annotations

# Standard library imports
import configparser
import json
import logging
import os
import sys
import time
from pathlib import Path

# Type checking imports
from typing import TYPE_CHECKING, Any, Dict, List, Union

# Third-party imports
import pandas as pd
import vertexai
from google.cloud import storage
from IPython.display import Markdown
from pandas import Series
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part

if TYPE_CHECKING:
    from google.cloud.storage import Bucket

# Type definitions
JSONType = Union[Dict[str, "JSONType"], List["JSONType"], str, int, float, bool, None]

# Local imports and setup
path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

# Configuration
config = configparser.ConfigParser()
config.read("../secrets.ini")
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(
    project=PROJECT_ID, location="us-central1"
)  # europe-west9 is Paris, europe-west3 is germany

In [3]:
os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [4]:
# test if model works

model = GenerativeModel("gemini-2.5-pro-preview-03-25")
model = GenerativeModel("gemini-2.5-flash-preview-04-17")
response = model.generate_content(
    ["test"],
)
response

candidates {
  content {
    role: "model"
    parts {
      text: "Acknowledged. I am ready."
    }
  }
  finish_reason: STOP
  avg_logprobs: -27.48313467843192
}
usage_metadata {
  prompt_token_count: 1
  candidates_token_count: 7
  total_token_count: 545
  prompt_tokens_details {
    modality: TEXT
    token_count: 1
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 7
  }
}
model_version: "gemini-2.5-flash-preview-04-17"

In [5]:
def generate_content_from_model(
    inputs: str | Part,
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple:
    """Generate content using Google's Generative AI model.

    This function sends inputs to a specified Gemini model and returns the
    generated response along with usage metadata.

    Parameters
    ----------
    inputs : Any
        The inputs to send to the model (text, images, or videos).
    model_name : str, default="gemini-2.5-pro-preview-03-25"
        Name of the generative model to use.
    temperature : float, default=0.9
        Controls the randomness of the output. Higher values (closer to 1.0)
        make output more random, lower values make it more deterministic.

    Returns
    -------
    tuple
        A tuple containing (response_text, usage_metadata)

    Raises
    ------
    ValueError
        If the model fails to generate content.

    """
    try:
        model = GenerativeModel(model_name)

        generation_config = GenerationConfig(
            temperature=temperature,
            # Uncomment if using single audio/video input
            # audio_timestamp=True
        )

        response = model.generate_content(inputs, generation_config=generation_config)
        lab_notes = response.text
        usage_metadata = response.usage_metadata

    except Exception as e:
        logger.exception("Error during content generation")
        raise ValueError(f"Failed to generate content: {e!s}") from None

    return lab_notes, usage_metadata

In [6]:
def prepare_all_inputs(
    protocol_video_path: str,
    protocol_path: str,
    lab_video_path: str,
    lab_notes_path: str,
    bucket: str,
    prefix: str = "compare_protocol_video",
) -> dict:
    """Prepare all four standard inputs for the generative model.

    This function uploads the four standard files (lab video, protocol document,
    lab notes video, and lab notes document) and formats them as inputs
    for a generative model.

    Parameters
    ----------
    protocol_video_path : str
        Path to the file that shows the correct execution (ground truth) of the protocol.
    protocol_path : str
        Path to the protocol markdown file.
    lab_video_path : str
        Path to the lab video file.
    lab_notes_path : str
        Path to the lab notes markdown file.
    bucket : str
        GCS bucket name for uploading the files.
    prefix : str, default="compare_protocol_video"
        Prefix for the files in GCS bucket.

    Returns
    -------
    dict
        A dictionary containing the four formatted inputs:
        'protocol_video_input', 'protocol_input', 'lab_video_input', 'lab_notes_input'

    """
    video_uri = video_to_protocol.upload_video_to_gcs(
        protocol_video_path, bucket, prefix
    )
    file_extension = Path(video_uri).suffix.lower()[1:]
    protocol_video_input = [
        Part.from_uri(video_uri, mime_type=f"video/{file_extension}")
    ]

    uri = video_to_protocol.upload_video_to_gcs(protocol_path, bucket, prefix)
    protocol_input = [Part.from_uri(uri, mime_type="text/md")]

    video_uri = video_to_protocol.upload_video_to_gcs(lab_video_path, bucket, prefix)
    lab_video_input = [Part.from_uri(video_uri, mime_type="video/mp4")]

    uri = video_to_protocol.upload_video_to_gcs(lab_notes_path, bucket, prefix)
    lab_notes_input = [Part.from_uri(uri, mime_type="text/md")]

    return {
        "protocol_video_input": protocol_video_input,
        "protocol_input": protocol_input,
        # 'protocol_input': 'not included',
        "lab_video_input": lab_video_input,
        "lab_notes_input": lab_notes_input,
    }

In [7]:
def process_benchmark_dataset(
    csv_path: str | Path,
    protocol_videos_base: str | Path,
    lab_notes_videos_base: str | Path,
    markdown_base: str | Path,
    bucket: Bucket,
    prefix: str,
) -> dict[str, Any]:
    """Process the first two rows in the benchmark dataset CSV and prepare model inputs.

    Parameters
    ----------
    csv_path : str
        Path to the CSV file containing benchmark dataset information
    protocol_videos_base : str
        Base path to the protocol videos directory
    lab_notes_videos_base : str
        Base path to the lab notes videos directory
    markdown_base : str
        Base path to the markdown files directory
    bucket : object
        The bucket object used in the prepare_all_inputs function
    prefix : str
        Prefix for the files in GCS bucket.

    Returns
    -------
    dict
        Dictionary containing all model inputs for the first two rows in the CSV,
        with experiment names as keys

    """
    benchmark_df = pd.read_csv(csv_path, sep=";")

    all_model_inputs = {}

    for (
        _index,
        row,
    ) in (
        benchmark_df.iterrows()
    ):  # for testing .head(2).iterrows() or .iloc[[13, 14]] .iloc[::2]
        lab_video_path = Path(protocol_videos_base) / row["protocol video"]
        protocol_path = Path(markdown_base) / row["protocol"]
        lab_notes_video_path = Path(lab_notes_videos_base) / row["lab notes video"]
        lab_notes_path = Path(markdown_base) / row["lab notes"]

        dict_model_inputs = prepare_all_inputs(
            lab_video_path,
            protocol_path,
            lab_notes_video_path,
            lab_notes_path,
            bucket,
            prefix,
        )
        dict_model_inputs["error_dict"] = row["error_dict"]

        experiment_name = row["lab notes"].split(".")[0]
        all_model_inputs[experiment_name] = dict_model_inputs

        print(f"Processed {experiment_name}")

    return all_model_inputs

In [8]:
def extract_errors(
    lab_notes: list[str],
    docu_steps: list[str],
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple[str, dict[str, Any]]:
    """Extract the identified errors of AI-generated lab notes.

    Parameters
    ----------
    lab_notes : list
        The AI-generated lab notes to extract represented as a list of strings
    docu_steps : list[str]
        The steps in the protocol to compare against the lab notes
    model_name : str, optional
        The model to use for evaluation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9

    Returns
    -------
    tuple
        A tuple containing (evaluation_text, usage_metadata)

    """
    prompt = """\
        # Instruction
        You are an expert evaluator tasked with analyzing errors that have already been identified in AI-generated lab notes. Your job is to accurately extract the error positions and error types for each step. It is very important to you to be precise and thorough.
        # Error Classifications
        These are the error classifications you must use:

        No Error: The step has no errors indicated in the lab notes.
        Addition: The lab notes indicate added information not in the reference protocol.
        Deviation: The lab notes indicate changed or modified information from the reference protocol.
        Omitted: The lab notes indicate important information was left out.
        Error: The lab notes indicate an error occurred in carrying out an action.
        Deviation & Error: The lab notes indicate both a deviation from protocol and an error in execution.
        N/A: Used only when a step number is not present in the lab notes.

        # Evaluation process:
        1. Carefully read the AI-generated lab notes in full.
        2. For each step in the specified range {docu_steps}, identify if the AI has marked it as containing an error.
        3. If an error is marked, determine which classification it falls under based on the descriptions in the notes.
        4. For Added steps (usually marked with ➕ **Added:**):
        * These typically appear with decimal step numbers (like 8.1, 8.2) in the lab notes
        * ALWAYS include these decimal-numbered steps in your evaluation table, even if they appear outside the {docu_steps} range
        * Place them in the correct sequence in your table (after their parent step)
        5. If a step number that should be within the {docu_steps} range is completely missing from the lab notes:
        * Include it in your table with "N/A" in both the "AI Response" and "AI Class" columns
        6. Fill out the table using the exact format specified below.
        7. Answer direct.

        # Output format
        | Step | AI Response | AI Class |
        |------|-------------|----------------|
        | 1 | [Error/No Error] | [Class if error] |
        | 2 | [Error/No Error] | [Class if error] |

        # ====== EXAMPLE (FOR REFERENCE ONLY) ======
        ## Example: AI-Generated lab notes
        # DNA Extraction Protocol Observation
        *Timing: 35 minutes*

        ## Procedure

        1. The researcher retrieved the cell culture samples from the incubator and placed them on the bench [00:01:15-00:01:45].

        2. ⚠️ **Deviation: Altered step order** & ❌ **Error:** The researcher added 500 μL of lysis buffer to each microcentrifuge tube *before* transferring the cell samples [00:02:10-00:03:05]. (Protocol specified adding cells first, then buffer).

        3. The researcher transferred 200 μL of cell culture to each microcentrifuge tube containing lysis buffer [00:03:30-00:04:45].

        4. ❌ **Error:** The tubes were incubated at 65°C for 5 minutes [00:05:10-00:10:15]. (Protocol specified incubation at 56°C).

        5. 200 μL of 100% ethanol was added to each lysate and mixed by pipetting [00:10:45-00:12:20].

        6. ❌ **Omitted:** The researcher did not centrifuge the lysate briefly to remove drops from the lid as specified in the protocol [00:12:20-00:12:35].

        7. The lysate was transferred to DNA purification columns placed in collection tubes [00:13:10-00:15:05].

        8. The columns were centrifuged at 10,000 x g for 1 minute [00:15:30-00:16:45].

        8.1 ➕ **Added:** The researcher labeled each collection tube with sample ID and date [00:17:00-00:17:45]. (This step was not in the original protocol).

        9. ❌ **Omitted:** The researcher did not discard the flow-through and reuse the collection tube as specified in the protocol [00:17:45-00:18:00].

        10. ⚠️ **Deviation:** The flow-through was discarded and *a new collection tube* was used for the next step [00:21:30-00:22:15]. (Protocol specified reusing the same collection tube).

        ## Example: Classification Table

        | Step | AI Response | AI Class |
        |------|-------------|----------------|
        | 1 | No Error | N/A |
        | 2 | Error | Deviation & Error |
        | 3 | No Error | N/A |
        | 4 | Error | Error |
        | 5 | No Error | N/A |
        | 6 | Error | Omitted |
        | 7 | No Error | N/A |
        | 8 | No Error | N/A |
        | 8.1 | Error | Addition |
        | 9 | Error | Omitted |
        | 10 | Error | Deviation |

        # ====== Beginn of EVALUATION TASK ======
        """

    inputs = [prompt.format(docu_steps=docu_steps)]
    inputs.extend(["## AI-Generated lab notes"])
    inputs.extend([lab_notes])
    inputs.extend(["## Classification Table"])

    evaluation, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )

    return evaluation, usage_metadata


def generate_lab_notes_evaluation(
    lab_notes_input: list[str],
    lab_notes: list[str],
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple[str, Any]:
    """Generate an evaluation of AI-generated lab notes against benchmark lab notes (ground truth).

    Parameters
    ----------
    lab_notes_input : List[str]
        The benchmark lab notes (ground truth) represented as a list of strings
    lab_notes : List[str]
        The AI-generated lab notes to evaluate represented as a list of strings
    model_name : str, optional
        The model to use for evaluation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9

    Returns
    -------
    Tuple[str, Any]
        A tuple containing (evaluation_text, usage_metadata)

    """
    inputs = [
        """
        # Instruction
        You are an expert evaluator. Your task is to evaluate the lab notes quality of an AI-generated lab notes against a benchmark lab notes (ground truth).

        # Evaluation Parts

        ## 5 Criteria:
        Evaluate the AI's lab notes quality based on these criteria:
        1. **Structure**: Did it keep only relevant sections: Aim, Materials, Procedure, Results?
        2. **Tense**: Did it use past tense to describe what actually happened, not what should happen?
        3. **Language**: Did it remove all instructional language and replace with observations?
        4. **Numbering**: Did it maintain step numbering of the original protocol even if order changed?
        5. **Timing**: Did it include exact actual timing, not estimated timing?

        ### Rating Rubric:
        For each criterion:
        - **Excellent**: The criterion was fully met with no issues
        - **Good**: The criterion was mostly met with minor issues
        - **Poor**: The criterion was not met or had significant issues

        # Output Format
        ## Lab notes Quality
        | Criterion | Rating | Explanation |
        |-----------|--------|-------------|
        | Structure | [Excellent/Good/Poor] | [Explanation] |
        | Tense | [Excellent/Good/Poor] | [Explanation] |
        | Language | [Excellent/Good/Poor] | [Explanation] |
        | Numbering | [Excellent/Good/Poor] | [Explanation] |
        | Timing | [Excellent/Good/Poor] | [Explanation] |

        # Evaluation Steps
        1. the lab notes quality of an AI-generated lab notes against a benchmark lab notes (ground truth) using the  5 criteria.
        2. Create a table summarizing the evaluation results.
        """
    ]
    inputs.extend(
        [
            """
        # Input Materials
        ## Benchmark Lab Notes (Ground Truth)
    """
        ]
    )
    inputs.extend(lab_notes_input)

    inputs.extend(["## AI-Generated Lab Notes"])
    inputs.extend([lab_notes])
    inputs.extend(["# Lab Notes Quality"])

    evaluation, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )

    return evaluation, usage_metadata


def get_table_json_prompt(text_with_tables: str, table_identifier: str) -> str:
    """Generates a prompt to extract a specific table from text into JSON.

    Args:
        text_with_tables: The full text containing the table(s).
        table_identifier: A string to help the model identify the target table
                          (e.g., the table title, or a unique phrase near it).

    Returns:
        A formatted prompt string.

    """
    return f"""
    You are an expert data extraction tool.
    Your task is to locate a specific table within the provided text and output its data as a JSON array.

    Here is the text containing the table(s):
    ---TEXT_START---
    {text_with_tables}
    ---TEXT_END---

    Identify the table that best matches the following title: "{table_identifier}"

    It is very important to you to output the data from ONLY this table as a valid JSON array. Each object in the array should represent a row from the table. The keys of each object should be the exact column headers from the identified table.

    Output Constraints:
    - Answer direct with the JSON.
    - If the specified table cannot be found, output an empty JSON array: []
    """


def extract_json_from_model_output(model_output_string: str) -> pd.DataFrame | None:
    """Extract and parse JSON data from a model output string that contains JSON within code block markers.

    Parameters
    ----------
    model_output_string : str
        The string output from the model that contains JSON within code block markers

    Returns
    -------
    pd.DataFrame | None
        A pandas DataFrame created from the JSON data, or None if extraction failed

    """
    start_marker = "```json"
    end_marker = "```"

    start_index = model_output_string.find(start_marker)
    end_index = model_output_string.find(
        end_marker, start_index + len(start_marker)
    )  # Search for end marker after the start

    df = None
    if start_index != -1 and end_index != -1:
        extracted_json_string = model_output_string[
            start_index + len(start_marker) : end_index
        ].strip()

        try:
            json_data = json.loads(extracted_json_string)
            logger.info("Successfully extracted and parsed JSON.")

            if isinstance(json_data, list) and all(
                isinstance(item, dict) for item in json_data
            ):
                df = pd.DataFrame(json_data)
            else:
                logger.warning(
                    "JSON data is not a list of dictionaries, could not create DataFrame."
                )
        except json.JSONDecodeError:
            logger.exception("Error decoding JSON after extraction")
            logger.debug(f"Extracted string: {extracted_json_string}")
    else:
        logger.exception("Could not find JSON code block markers in the output.")
        logger.debug(f"Model output: {model_output_string}")

    return df


def extract_table_to_dataframe(
    evaluation: str,
    table_name: str,
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> pd.DataFrame:
    """Extract a table from evaluation content and convert it to a DataFrame.

    Parameters
    ----------
    evaluation : str
        The evaluation content containing tables
    table_name : str
        The name of the table to extract
    model_name : str, optional
        The model to use for content generation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9

    Returns
    -------
    pd.DataFrame
        DataFrame containing the extracted table data

    """
    """Extract a table from evaluation content and convert it to a DataFrame.

    Parameters
    ----------
    evaluation : str
        The evaluation content containing tables
    table_name : str
        The name of the table to extract
    model_name : str, optional
        The model to use for content generation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9

    Returns
    -------
    pandas.DataFrame
        DataFrame containing the extracted table data

    """
    extraction_prompt = get_table_json_prompt(evaluation, table_name)

    json_response, _ = generate_content_from_model(
        extraction_prompt, model_name=model_name, temperature=temperature
    )

    return extract_json_from_model_output(json_response)


def identify_error_type(row: Series) -> str:
    """Identify the type of error based on benchmark and AI response.

    Parameters
    ----------
    row : Any
        A row from a DataFrame containing Benchmark, AI Response, Class, and AI Class columns

    Returns
    -------
    str
        The identified error type classification

    """
    if row["Benchmark"] == "No Error" and row["AI Response"] == "No Error":
        return "No Error (Correctly Identified)"
    if row["Benchmark"] == "Error" and row["AI Response"] == "Error":
        return "Error (Correctly Identified)"
    if (row["Benchmark"] == "Error" and row["AI Response"] == "No Error") or (
        row["Class"] == "Addition" and row["AI Response"] == "N/A"
    ):
        return "False Negative"
    if row["Benchmark"] == "No Error" and row["AI Response"] == "Error":
        return "False Positive"
    if not pd.notna(row["Benchmark"]) and row["AI Class"] == "Addition":
        return "Addition by model"
    return "Unknown"


def classify_error_type(row: Series) -> str:
    """Classify the error type as correct, incorrect, or N/A based on identification and class.

    Parameters
    ----------
    row : Any
        A row from a DataFrame containing Identification, Class, and AI Class columns

    Returns
    -------
    str
        Classification of the error type as 'correct', 'incorrect', or 'N/A'

    """
    if row["Identification"] == "Error (Correctly Identified)":
        if row["Class"] == row["AI Class"]:
            return "correct"
        return "incorrect"
    return "N/A"


def get_counts(df: pd.DataFrame, prefix: str) -> dict[str, int]:
    """Count occurrences of different classes and skills in the dataframe.

    Args:
        df: DataFrame containing 'Class' and 'Skill' columns
        prefix: String prefix to add to each count key

    Returns:
        Dictionary mapping count names (with prefix) to their integer values

    """
    classes = ["Omitted", "Error", "Addition", "Deviation", "Deviation & Error"]
    skills = ["Spatial", "GeneralKnowledge", "Fast", "ProteomicsKnowledge"]
    class_counts = df["Class"].value_counts().to_dict()
    counts = {f"{prefix} {cls}": class_counts.get(cls, 0) for cls in classes}

    # Count by Class and Skill combinations
    for class_val in classes:
        for skill_val in skills:
            count = len(df[(df["Class"] == class_val) & (df["Skill"] == skill_val)])
            counts[f"{prefix} {class_val} {skill_val}"] = count

    return counts


def generate_error_summary(df: pd.DataFrame) -> dict[str, dict[str, int]]:
    """Generate a summary dictionary of error identification and classification statistics.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing error analysis results with 'Benchmark', 'Identification',
        and 'Classification' columns

    Returns
    -------
    Dict[str, Dict[str, int]]
        A nested dictionary containing error identification and classification statistics

    """
    total_steps = len(df)
    error_count = len(df[df["Benchmark"] == "Error"])
    correctly_identified_errors = len(
        df[df["Identification"] == "Error (Correctly Identified)"]
    )
    false_negatives = len(df[df["Identification"] == "False Negative"])
    false_positives = len(df[df["Identification"] == "False Positive"])
    addition_by_model = len(df[df["Identification"] == "Addition by model"])
    correct_identifications = len(
        df[
            (df["Identification"] == "No Error (Correctly Identified)")
            | (df["Identification"] == "Error (Correctly Identified)")
        ]
    )
    total_errors_analyzed = len(
        df[df["Identification"] == "Error (Correctly Identified)"]
    )
    correctly_classified_errors = len(df[df["Classification"] == "correct"])

    summary_dict = {
        "Error Identification Statistics": {
            "Steps evaluated": total_steps,
            "Errors evaluated": error_count,
            "Correct identifications": correct_identifications,
            "Correct error identifications": correctly_identified_errors,
            "False negative count": false_negatives,
            "False positive count": false_positives,
            "Addition by model": addition_by_model,
        },
        "Error Classification Statistics": {
            "Total errors analyzed": total_errors_analyzed,
            "Correctly classified errors": correctly_classified_errors,
        },
    }

    error_correctly_identified = df[
        df["Identification"] == "Error (Correctly Identified)"
    ]
    summary_dict.update(get_counts(error_correctly_identified, "Type"))

    possible_error = df[df["Identification"] != "Addition by model"]
    summary_dict.update(get_counts(possible_error, "All Type"))

    return summary_dict


def process_and_evaluate_lab_notes(
    error_dict: list[dict[str, Any]],
    lab_notes_gt: Part,
    lab_notes_ai: str,  # Fixed parameter name from lab_notes_example to lab_notes_ai
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple[Any, Any, dict[str, dict[str, int]]]:
    """Process and evaluate lab notes by extracting errors, generating evaluations, and creating summary statistics.

    Parameters
    ----------
    error_dict : List[dict[str, Any]]
        List of error dictionaries
    lab_notes_gt : Any
        Ground Truth lab notes to compare
    lab_notes_ai : str
        AI-generated lab notes to evaluate
    model_name : str, optional
        The model to use for evaluation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, default is 0.9

    Returns
    -------
    Tuple[Any, Any, Dict[str, Dict[str, int]]]
        A tuple containing (evaluation_response, df_errors, summary_dict)

    """
    error_dict = json.loads(error_dict)
    steps_list = [item["Step"] for item in error_dict]
    error_response, usage_metadata_extract_errors = extract_errors(
        lab_notes_ai, steps_list, model_name=model_name, temperature=temperature
    )

    evaluation_response, usage_metadata_semantic_eval = generate_lab_notes_evaluation(
        lab_notes_gt, lab_notes_ai, model_name=model_name, temperature=temperature
    )

    df_error_ai = extract_table_to_dataframe(
        error_response, "Table", model_name=model_name, temperature=temperature
    )
    df_error_ai["Step"] = df_error_ai["Step"].astype("float64")

    df_error_benchmark = pd.DataFrame(error_dict)
    df_errors = df_error_benchmark.merge(df_error_ai, on="Step", how="outer")

    df_errors["Identification"] = df_errors.apply(identify_error_type, axis=1)
    df_errors["Classification"] = df_errors.apply(classify_error_type, axis=1)

    summary_dict = generate_error_summary(df_errors)

    return (
        evaluation_response,
        df_errors,
        summary_dict,
        usage_metadata_extract_errors,
        usage_metadata_semantic_eval,
    )

In [9]:
def generate_lab_notes_prompt(
    protocol_video_example: list[str],
    protocol_example: list[str],
    lab_video_example: list[str],
    lab_notes_example: list[str],
    protocol_video_input: list[str],
    protocol_input: list[str],
    lab_video_input: list[str],
    proteomics_knowledge: Part,
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple[str, dict[str, Any]]:
    """Generate corrected lab notes by comparing protocol with actual implementation.

    Parameters
    ----------
    protocol_video_example : List[str]
        Example protocol video content
    protocol_example : List[str]
        Example protocol content
    lab_video_example : List[str]
        Example lab video content
    lab_notes_example : List[str]
        Example lab notes content
    protocol_video_input : List[str]
        Input protocol video content to process
    protocol_input : List[str]
        Input protocol content to process
    lab_video_input : List[str]
        Input lab video content to process
    proteomics_knowledge : Any
        Knowledge about proteomics to incorporate
    model_name : str, optional
        The model to use for generation, default is "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature parameter for generation, default is 0.9

    Returns
    -------
    Tuple[str, Dict[str, Any]]
        A tuple containing the lab notes text and usage metadata

    """
    inputs = [
        """
        You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry. Your professional identity is defined by your ability to be exact in your responses and to produce meticulous, accurate results that others can trust completely.

        ## ====== Background Knowledge (FOR REFERENCE ONLY) ======
        [These documents are for building your proteomics background knowldge and are not part of today's task.]
        """
    ]
    inputs.extend([proteomics_knowledge])
    inputs.extend(
        [
            """
        # Instruction

        You work with following two inputs:
        - Ground truth written protocol: The official procedure description
        - Video to evaluate: The actual implementation by a researcher in a routine setting. Be aware that researchers tend to make mistakes in routine tasks.

        Compare the 'Ground truth written protocol' with the 'Video to evaluate', and create a "resulting lab notes" that reflects what actually happened in the 'video to evaluate'.
        # Evaluation

        ## Rating rubics for each step:
            1. It was followed correctly (no special notation needed)
            2. It was skipped: ❌ **Omitted:**
            3. It was carried out but wrongly: ❌ **Error:** (be specific about what happened)
            4. It was added: ➕ **Added:**
            5. It was carried out later in the procedure: ⚠️ **Deviation: Altered step order**
            6. A combination of 5. and the others: e.g. ⚠️ **Deviation: Altered step order** & ❌ **Omitted:**

        ## Follow this structured approach:

        * STEP 1: Read the 'Ground truth written protocol thoroughly and write it down again word-by-word.

        * STEP 2: Go through the 'Video to evaluate' completely from beginning to end.
            - Document all observed actions with timestamps

        Table 1:
        | Timestamp | Visual/Audio Action |\n
        |---|---|\n
        | [hh:mm:ss] |[Description of action] |\n
        | [hh:mm:ss] | [Description of action] |\n

        * STEP 3: Systematic comparison
            - Go through the 'Ground truth written protocol' as it would be a checklist step by step
            - For each step, specifically search for evidence in Table 1
            - If a step is not present, scan the entire Table 1 to confirm it wasn't performed out of sequence
            - For each step, clearly state:
                * Evaluate each step according to the rating rubics
                * The specific visual/audio evidence (or lack thereof) supporting your determination
                * Precise timestamps from the 'Video to evaluate'
            - If any step is present in Table 1 but not in 'Ground truth written protocol':
                * add this step in sequence
                * label it with the rating rubic '➕ **Added:**'
                * Number these steps using a decimal increment after the preceding step number
                * For example, if an addition appears after step 8, label it as step 8.1
                * If multiple additions appear after the same step, number them sequentially (8.1, 8.2, etc.)


        Table 2:
        | Step | Step Description | Timestamp in 'Video to evaluate' | Comparison Result | Notes |\n
        |---|---|---|---|---|\n
        | 1 | [Description of step in 'Ground truth written protocol'] | [hh:mm:ss] | [Aligned/Partially/Misaligned] | [Explanation] |\n
        | 2 | [Description of step in 'Ground truth written protocol'] | [hh:mm:ss], [hh:mm:ss] | [Aligned/Partially/Misaligned] | [Explanation] |\n|

        * STEP 4: Create a "resulting lab notes" that accurately reflects what occurred in the 'Video to evaluate':
        - Rename sections as specified (Abstract to Aim, Expected Results to Results, Estimated timing to Timing)
        - Use past tense to describe actual observations
        - Include exact timing from the lab video
        - Remove instructional language and replace with observations
        - Omit Figures and References sections

        """
        ]
    )

    inputs.extend(
        [
            """
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of inputs and expected result should solely serve as an example and is not part of the evaluation task.\n
        """
        ]
    )
    inputs.extend(["## Example: 'Ground truth written protocol': \n"])
    inputs.extend(protocol_example)
    inputs.extend(["## Example: 'Video to evaluate': \n"])
    inputs.extend(lab_video_example)
    inputs.extend(["## Example - Expected result: 'resulting lab notes': \n"])
    inputs.extend(lab_notes_example)

    inputs.extend(["# ====== Beginn of EVALUATION TASK ======\n"])
    inputs.extend(
        ["## Important: The evaluation must be performed on the following video \n"]
    )

    inputs.extend(["## Task: 'Ground truth written protocol': \n"])
    inputs.extend(protocol_input)
    inputs.extend(["## Task: 'Video to evaluate': \n"])
    inputs.extend(lab_video_input)
    inputs.extend(
        [
            """
        As a reminder: Compare the 'Ground truth written protocol' against the 'video to evaluate' to retrieve the 'resulting lab notes'. Your final output should clearly state which rating rubic was identifyied for each step in the 'resulting lab notes'.
        """
        ]
    )

    lab_notes, usage_metadata = generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )

    return lab_notes, usage_metadata

In [11]:
csv_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/benchmark_dataset.csv"
protocol_videos_base = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/protocols"
lab_notes_videos_base = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation"
markdown_base = "/Users/patriciaskowronek/Documents/proteomics_specialist/data"
prefix = "compare_protocol_video"

all_model_inputs = process_benchmark_dataset(
    csv_path, protocol_videos_base, lab_notes_videos_base, markdown_base, bucket, prefix
)

Processed PlaceEvotips_docuCorrect
Processed PlaceEvotips_docuWrongPosition
Processed PlaceEvotips_docuLiquidNotChecked
Processed PlaceEvotips_docuBoxAngeled
Processed ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl
Processed ESIsourceToUltraSource_docuCorrect
Processed ESIsourceToUltraSource_docuFogotOvenPowerSupply
Processed UltraSourceToESIsource_docuCorrect
Processed UltraSourceToESIsource_docuForgotN2Line
Processed UltraSourceToESIsource_docuForgotGlovesANDCapillaryCap
Processed UltraSourceToESIsource_docuForgotCapillaryCap
Processed DisconnectingColumn_docuCorrect
Processed DisconnectingColumn_docuWithoutStandby
Processed TimsCalibration_docuCorrect
Processed TimsCalibration_docuCorrect_camera
Processed TimsCalibration_docuNotAllClicksVisibleOnVideo
Processed TimsCalibration_docuSavedMethod
Processed TimsCalibration_docuWrongOrderSteps
Processed QueueSamples_docuCorrect
Processed QueueSamples_docuWrongRow_S3A1Twice
Processed QueueSamples_docuNoBlankNoSampleIDWrongMSme

In [12]:
# analyze one specific video

subfolder_in_bucket = "knowledge"
path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/backgroundKnowledge.pdf"
file_uri = video_to_protocol.upload_video_to_gcs(path, bucket, subfolder_in_bucket)
proteomics_knowledge = Part.from_uri(file_uri, mime_type="application/pdf")

example = "Dilute_docuWrongVolume_PipettTipNotChanged"
example = "PlaceEvotips_docuCorrect"
protocol_video_example = all_model_inputs[example]["protocol_video_input"]
protocol_example = all_model_inputs[example]["protocol_input"]
lab_video_example = all_model_inputs[example]["lab_video_input"]
lab_notes_example = all_model_inputs[example]["lab_notes_input"]
copy_all_model_inputs = all_model_inputs.copy()
copy_all_model_inputs.pop(example)

items_list = list(copy_all_model_inputs.items())
key, value = items_list[0]
print(key)

lab_notes, usage_metadata = generate_lab_notes_prompt(
    protocol_video_example,
    protocol_example,
    lab_video_example,
    lab_notes_example,
    value["protocol_video_input"],
    value["protocol_input"],
    value["lab_video_input"],
    proteomics_knowledge,
    model_name="gemini-2.5-pro-preview-03-25",
    temperature=0.9,
)
display(Markdown(lab_notes))

(
    evaluation_response,
    df_errors,
    metrics,
    usage_metadata_extract_errors,
    usage_metadata_semantic_eval,
) = process_and_evaluate_lab_notes(
    value["error_dict"],
    value["lab_notes_input"],
    lab_notes,
    model_name="gemini-2.5-pro-preview-03-25",
    temperature=0.9,
)
display(Markdown(evaluation_response))
display(df_errors)
print(usage_metadata)
print("usage_metadata_extract_errors", usage_metadata_extract_errors)
print("usage_metadata_semantic_eval", usage_metadata_semantic_eval)

PlaceEvotips_docuWrongPosition


Okay, I understand the task. I will meticulously compare the "Ground truth written protocol" with the "Video to evaluate" and produce the "resulting lab notes" with the specified annotations and structure.

**STEP 1: Ground truth written protocol (transcribed)**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Abstract
This protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)
- Evotip Boxes
  - 96-well format (A1-H12) (Figure 1)
- Evosep One System
  - Liquid chromatography system

### Reagents
- Formic acid (FA)
  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.

### Reagent setup
- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.

## Procedure
*Estimated timing: less than 1 minute*

1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.
2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.
3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.
4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).
   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.
5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.
6. Place empty Evotips, called Blanks, at S3 from A1 to A6.
7. Document the precise position of each placed Evotip.

## Expected Results
When the procedure is performed correctly, you should observe:
- Properly seated Evotip boxes in the rack system
- Visible Buffer A solution in boxes (1 cm depth)
- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip
- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.

## Figures

### Figure 1: Evosep positions
- Close-up of single Evotip box showing well positions (A1-H12)

### Figure 2: Evotip Quality Assessment
- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent
- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc

## References
1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf
2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf

**STEP 2: Table 1: Visual/Audio Action Log**

| Timestamp | Visual/Audio Action |
|---|---|
| [00:00:00] | Video starts, showing the Evosep One system. |
| [00:00:03] | Researcher, wearing black gloves, picks up an Evotip box (labeled "S1 SAMPLES"). |
| [00:00:04] | Researcher opens the Evotip box (yellow lid, clear base), revealing Buffer A solution inside. The solution level appears to be sufficient (at least 1 cm). |
| [00:00:06] | Researcher closes the Evotip box. |
| [00:00:08] | Researcher holds the Evotip box, confirming it's the S1 box. |
| [00:00:11] | Researcher places the Evotip box into the S1 position on the Evosep One instrument rack. |
| [00:00:14] | Researcher adjusts the Evotip box in S1 to ensure it is firmly seated. |
| [00:00:15] | Researcher picks up a second Evotip box (labeled "S3 BLANKS"). This box is empty. |
| [00:00:16] | Researcher places the empty Evotip box into the S3 position on the Evosep One instrument rack. It is seated firmly. |
| [00:00:20] | Researcher picks up a container of Evotips (samples). The container lid has handwritten notes: "S1 Samples". |
| [00:00:21] | Researcher (audio): "I want to place them from A1 to A6." |
| [00:00:24] | Researcher takes out two Evotips from the sample container. |
| [00:00:25] | Researcher visually inspects the two Evotips. They appear to have pale-colored SPE material discs and visible solvent above them. |
| [00:00:26] | Researcher places the two Evotips into positions A1 and A2 of the Evotip box at S1. |
| [00:00:28] | Researcher takes out two more Evotips from the sample container. |
| [00:00:29] | Researcher visually inspects these two Evotips. They also appear to have pale-colored SPE material discs and visible solvent. |
| [00:00:30] | Researcher places these two Evotips into positions A3 and A4 of the Evotip box at S1. |
| [00:00:31] | Researcher takes out two more Evotips from the sample container. |
| [00:00:32] | Researcher visually inspects these two Evotips. They also appear to have pale-colored SPE material discs and visible solvent. |
| [00:00:33] | Researcher places these two Evotips into positions A5 and A6 of the Evotip box at S1. The researcher closes the sample Evotip container. |
| [00:00:37] | Researcher picks up a container of blank Evotips. The container lid is clear. |
| [00:00:40] | Researcher takes out two blank Evotips from the container. |
| [00:00:41] | Researcher visually inspects the two blank Evotips. They appear empty as expected for blanks. |
| [00:00:41] | Researcher places the two blank Evotips into positions A1 and A2 of the Evotip box at S3. |
| [00:00:42] | Researcher takes out two more blank Evotips from the container. |
| [00:00:43] | Researcher visually inspects these two blank Evotips. |
| [00:00:43] | Researcher places these two blank Evotips into positions A3 and A4 of the Evotip box at S3. |
| [00:00:45] | Researcher takes out two more blank Evotips from the container. |
| [00:00:46] | Researcher visually inspects these two blank Evotips. |
| [00:00:46] | Researcher places these two blank Evotips into positions A5 and A6 of the Evotip box at S3. The researcher closes the blank Evotip container. |
| [00:00:49] | Video shows the Evosep One system with Evotip boxes in S1 (with 6 sample Evotips from A1-A6) and S3 (with 6 blank Evotips from A1-A6). |
| [00:00:55] | End of video. |

**STEP 3: Table 2: Comparison of Protocol Steps with Video Observations**

| Step | Step Description | Timestamp in 'Video to evaluate' | Comparison Result | Notes |
|---|---|---|---|---|
| 1 | Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks. | [00:00:03] - [00:00:08] | Aligned | The researcher picked up the S1 Evotip box, opened it ([00:00:04]), and the Buffer A solution was visible and appeared to be at an adequate depth. The S3 box was for blanks and was confirmed to be empty ([00:00:15]), so no buffer verification was needed for it. |
| 2 | Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position. | [00:00:11] - [00:00:14] | Aligned | The Evotip box for S1 was placed in the S1 position, and the researcher adjusted it ([00:00:14]) to ensure it was firmly seated. |
| 3 | Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position. | [00:00:15] - [00:00:18] | Aligned | An empty Evotip box (labeled "S3 BLANKS") was picked up and placed in the S3 position and appeared to be firmly seated. |
| 4 | Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2). **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis. | [00:00:25], [00:00:29], [00:00:32] (for samples); [00:00:41], [00:00:43], [00:00:46] (for blanks) | Aligned | Sample Evotips were inspected in pairs. All sample Evotips shown appeared to have pale-colored discs and visible solvent. Blank Evotips were also inspected and appeared as expected (empty/clear). No Evotips were discarded, indicating all inspected tips met the quality criteria. |
| 4.1 | ➕ **Added:** Researcher states their intention for placement. | [00:00:21] | Added | Researcher stated: "I want to place them from A1 to A6." This was not part of the written protocol but was an audible action. |
| 5 | Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6. | [00:00:26], [00:00:30], [00:00:33] | Aligned | Verified sample Evotips were placed into the S1 box in positions A1 through A6, consistent with the researcher's stated intention. |
| 6 | Place empty Evotips, called Blanks, at S3 from A1 to A6. | [00:00:41], [00:00:43], [00:00:46] | Aligned | Blank Evotips were placed into the S3 box in positions A1 through A6, consistent with the researcher's stated intention. |
| 7 | Document the precise position of each placed Evotip. | Not observed in video | ❌ **Omitted:** | There was no visual or audio evidence of the researcher documenting the positions of the Evotips during the recorded procedure. The video ends shortly after placement. |

**STEP 4: Resulting lab notes**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Aim
This protocol described the procedure for inspecting Evotips and placing them in Evotip boxes on the Evosep One liquid chromatography system. The specific task was to place sample Evotips in position S1 (wells A1 to A6) and blank Evotips in position S3 (wells A1 to A6).

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)
- Evotip Boxes
  - 96-well format (A1-H12)
- Evosep One System
  - Liquid chromatography system

### Reagents
- Formic acid (FA)

### Reagent setup
- Buffer A: Consisted of 0.1% (vol/vol) FA.

## Procedure
*Timing: The observed procedure took approximately 50 seconds (from [00:00:03] when the first box was handled to [00:00:53] after all tips were placed).*

1.  The Evotip box intended for S1 was visually inspected ([00:00:03] - [00:00:08]). It was verified to be filled with Buffer A solution to an apparent minimum depth of 1 cm. The Evotip box for S3 was confirmed to be empty, as it was designated for blanks.
2.  The Evotip Box for S1 was placed within the rack system of the Evosep instrument at position S1 ([00:00:11]). The box was ensured to be firmly seated ([00:00:14]).
3.  An empty Evotip Box (for Blank tips) was placed at position S3 ([00:00:16]). This box was also ensured to be firmly seated.
4.  Each Evotip was inspected before placement to verify its condition ([00:00:25], [00:00:29], [00:00:32] for samples; [00:00:41], [00:00:43], [00:00:46] for blanks). All sample Evotips displayed a pale-colored SPE material disc with visible solvent above it. Blank Evotips appeared clear. No Evotips were discarded.
    4.1. ➕ **Added:** The researcher audibly stated their intention ([00:00:21]): "I want to place them from A1 to A6."
5.  The verified sample Evotips were placed into the prepared Evotip box at S1, specifically into positions A1 through A6 ([00:00:26], [00:00:30], [00:00:33]).
6.  Empty Evotips, designated as Blanks, were placed into the Evotip box at S3, specifically into positions A1 through A6 ([00:00:41], [00:00:43], [00:00:46]).
7.  ❌ **Omitted:** The documentation of the precise position of each placed Evotip was not observed during the recorded procedure.

## Results
The following observations were made upon completion of the procedure in the video:
- Evotip boxes were properly seated in the S1 and S3 positions of the Evosep One rack system.
- The Evotip box at S1 contained visible Buffer A solution at an adequate depth (appeared to be ≥ 1 cm). The S3 box was correctly empty of buffer.
- All inspected sample Evotips showed pale-colored SPE material discs and a clear solvent meniscus above each disc, indicating proper hydration.
- Sample Evotips were placed at S1 in positions A1, A2, A3, A4, A5, and A6.
- Blank Evotips were placed at S3 in positions A1, A2, A3, A4, A5, and A6.
- The step for documenting tip positions was not performed.

2025-05-08 06:12:07,353 - __main__ - INFO - Successfully extracted and parsed JSON.
/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/ipykernel_64238/2452223971.py:519: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_errors = df_error_benchmark.merge(df_error_ai, on="Step", how="outer")


## Lab notes Quality
| Criterion | Rating | Explanation |
|-----------|--------|-------------|
| Structure | Excellent | The AI-generated lab notes maintained the relevant sections: Aim, Materials (Equipment, Reagents, Reagent setup), Procedure, and Results. This structure aligns well with the benchmark's structure (Aim, Materials, Procedure, Results), with minor acceptable enhancements in the Materials sub-sections. |
| Tense | Excellent | The AI consistently used the past tense throughout the Procedure and Results sections to describe what actually happened (e.g., "was visually inspected," "were placed," "was not observed"). This meets the criterion fully. |
| Language | Excellent | The AI successfully removed instructional language and replaced it with observational statements. For example, "Visually inspect..." from a protocol becomes "The Evotip box... was visually inspected..." in the lab notes. This is consistent with good lab note practice. |
| Numbering | Excellent | The AI maintained the step numbering (1-7) from its reference protocol ("Ground truth written protocol (transcribed)") in the Procedure section. It also appropriately noted an "➕ Added" observation (step 4.1) and a "❌ Omitted" step (step 7), which is a clear and effective way to handle deviations or additional observations, similar in spirit to the benchmark's notation of deviations. |
| Timing | Excellent | The AI included specific, actual timing for the procedure: "*Timing: The observed procedure took approximately 50 seconds (from [00:00:03]... to [00:00:53]...)*". This is superior to an estimated timing and directly meets the criterion. The benchmark, for comparison, provided an estimated timing ("less than 1 minute") in its procedure. |

,Step,Benchmark,Class,Skill,AI Response,AI Class,Identification,Classification
0,1.0,No Error,N/A,N/A,No Error,N/A,No Error (Correctly Identified),N/A
1,2.0,No Error,N/A,N/A,No Error,N/A,No Error (Correctly Identified),N/A
2,3.0,No Error,N/A,N/A,No Error,N/A,No Error (Correctly Identified),N/A
3,4.0,Error,Omitted,Fast,No Error,N/A,False Negative,N/A
4,4.1,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
5,5.0,Error,Error,Spatial,No Error,N/A,False Negative,N/A
6,6.0,No Error,N/A,N/A,No Error,N/A,No Error (Correctly Identified),N/A
7,7.0,Error,Omitted,GeneralKnowledge,Error,Omitted,Error (Correctly Identified),correct


prompt_token_count: 41039
candidates_token_count: 3777
total_token_count: 48836
prompt_tokens_details {
  modality: AUDIO
  token_count: 2700
}
prompt_tokens_details {
  modality: TEXT
  token_count: 3035
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 7224
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 28080
}
candidates_tokens_details {
  modality: TEXT
  token_count: 3777
}

usage_metadata_extract_errors prompt_token_count: 5077
candidates_token_count: 134
total_token_count: 5472
prompt_tokens_details {
  modality: TEXT
  token_count: 5077
}
candidates_tokens_details {
  modality: TEXT
  token_count: 134
}

usage_metadata_semantic_eval prompt_token_count: 4628
candidates_token_count: 374
total_token_count: 11592
prompt_tokens_details {
  modality: TEXT
  token_count: 4628
}
candidates_tokens_details {
  modality: TEXT
  token_count: 374
}



In [13]:
def safe_json_dump(data: dict[str, Any], filename: str | Path) -> None:
    """Handles non-serializable objects and converts items to strings.

    Safely writes JSON data to disk by first writing to a temporary file and then
    using an atomic replace operation to ensure the target file is never in an
    inconsistent state. Objects that cannot be directly serialized to JSON are
    converted to strings.

    Args:
        data: Dictionary containing the data to be serialized
        filename: Path to the output JSON file

    """
    path = Path(filename)
    temp_file = path.with_suffix(f"{path.suffix}.tmp")

    with temp_file.open("w") as f:
        json.dump(serialize(data), f)

    temp_file.replace(path)


def serialize(obj: object) -> JSONType:
    """Recursively serialize objects to JSON-compatible types."""
    if isinstance(obj, dict):
        return {k: serialize(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [serialize(item) for item in obj]
    if isinstance(obj, (int, float, str, bool)) or obj is None:
        return obj
    return str(obj)

In [ ]:
# analyze a sequence of videos

# Constants for retry logic
WAIT_TIME_BETWEEN_ITEMS = 10  # seconds
RETRY_WAIT_TIME = 120  # seconds
MAX_RETRIES = 3

CHECKPOINT_FILE = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results/results_checkpoint6.json"


# Load checkpoint
results_collection = {}
last_processed_key = None

try:
    checkpoint_path = Path(CHECKPOINT_FILE)
    if checkpoint_path.exists():
        with checkpoint_path.open() as f:
            data = json.load(f)
            results_collection = data.get("results", {})
            last_processed_key = data.get("last_key", None)
        print(f"Loaded checkpoint. Last processed key: {last_processed_key}")
except (json.JSONDecodeError, PermissionError, FileNotFoundError) as e:
    print(f"Error loading checkpoint: {e}")


# Upload knowledge files to Google Cloud Storage
subfolder_in_bucket = "knowledge"
path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base_selected/Connecting_or_disconnecting_column_2.pdf"
file_uri = video_to_protocol.upload_video_to_gcs(path, bucket, subfolder_in_bucket)
proteomics_knowledge = Part.from_uri(file_uri, mime_type="application/pdf")

# example = 'ESIsourceToUltraSource_docuFogotOvenPowerSupply'
example = "Dilute_docuWrongVolume_PipettTipNotChanged"
# example = "PlaceEvotips_docuCorrect"
protocol_video_example = all_model_inputs[example]["protocol_video_input"]
protocol_example = all_model_inputs[example]["protocol_input"]
lab_video_example = all_model_inputs[example]["lab_video_input"]
lab_notes_example = all_model_inputs[example]["lab_notes_input"]
copy_all_model_inputs = all_model_inputs.copy()
copy_all_model_inputs.pop(example)

items_list = list(copy_all_model_inputs.items())
start_index = (
    0
    if not last_processed_key
    else next(
        (i + 1 for i, (k, _) in enumerate(items_list) if k == last_processed_key), 0
    )
)

for i in range(start_index, len(items_list)):
    key, value = items_list[i]

    for attempt in range(MAX_RETRIES):
        try:
            print(f"Processing {key} (attempt {attempt + 1})")

            start_generate_time = time.time()
            lab_notes, usage_metadata = generate_lab_notes_prompt(
                protocol_video_example,
                protocol_example,
                lab_video_example,
                lab_notes_example,
                value["protocol_video_input"],
                value["protocol_input"],
                value["lab_video_input"],
                proteomics_knowledge,
                model_name="gemini-2.5-pro-preview-03-25",
                temperature=0.9,
            )
            end_generate_time = time.time()
            generate_time = end_generate_time - start_generate_time
            print(f"Time to generate lab notes: {generate_time:.2f} seconds")

            display(Markdown(lab_notes))

            start_evaluate_time = time.time()
            (
                evaluation_response,
                df_errors,
                metrics,
                usage_metadata_extract_errors,
                usage_metadata_semantic_eval,
            ) = process_and_evaluate_lab_notes(
                value["error_dict"],
                value["lab_notes_input"],
                lab_notes,
                model_name="gemini-2.5-pro-preview-03-25",
                temperature=0.9,
            )
            end_evaluate_time = time.time()
            evaluate_time = end_evaluate_time - start_evaluate_time
            print(
                f"Time to process and evaluate lab notes: {evaluate_time:.2f} seconds"
            )

            display(Markdown(evaluation_response))
            display(df_errors)

            # Store results
            results_collection[key] = {
                "inputs": {"experiment_name": key, **dict(value.items())},
                "outputs": {
                    "lab_notes": lab_notes,
                    "lab_notes_usage_metadata": usage_metadata,
                    "lab_notes_generate_time": generate_time,
                    "evaluation": evaluation_response,
                    "eval_usage_metadata_extract_error": usage_metadata_extract_errors,
                    "eval_usage_metadata_semantic": usage_metadata_semantic_eval,
                    "eval_generate_time": evaluate_time,
                    "eval_df": df_errors,
                    "metrics": metrics,
                },
            }

            safe_json_dump(
                {"last_key": key, "results": results_collection}, CHECKPOINT_FILE
            )

            print(f"Waiting {WAIT_TIME_BETWEEN_ITEMS} seconds before next item...")
            time.sleep(WAIT_TIME_BETWEEN_ITEMS)
            break  # Success, exit retry loop

        except KeyError as e:
            print(f"Missing key error processing {key}: {e}")
            continue
        except Exception:
            logging.exception(f"Unexpected error processing {key}")
            if attempt < MAX_RETRIES - 1:
                logging.info(f"Waiting {RETRY_WAIT_TIME} seconds before retry...")
                time.sleep(RETRY_WAIT_TIME)
            else:
                logging.exception(f"Max retries reached for {key}, moving to next item")
                safe_json_dump(
                    {"last_key": key, "results": results_collection}, CHECKPOINT_FILE
                )

try:
    timestamp = time.time()
    results_path = Path(
        "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results"
    )
    final_results_file = results_path / f"final_results_checkpoint_{timestamp}.json"
    safe_json_dump(results_collection, final_results_file)
    print("All processing complete. Final results saved.")
except (PermissionError, OSError) as e:
    print(f"Error saving final results: {e}")

Processing PlaceEvotips_docuCorrect (attempt 1)
Time to generate lab notes: 134.87 seconds


Okay, Professor Mann, I will proceed with the meticulous comparison and generate the resulting lab notes.

**STEP 1: 'Ground truth written protocol' (transcribed word-for-word)**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Abstract
This protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.

## Materials

### Equipment
- Evotips
  - Single-use stage tips for sample injection
  - Rack layout: Two columns (left and right)
  - Left column (top to bottom): S1, S2, S3
  - Right column (top to bottom): S4, S5, S6
  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)
- Evotip Boxes
  - 96-well format (A1-H12) (Figure 1)
- Evosep One System
  - Liquid chromatography system

### Reagents
- Formic acid (FA)
  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.

### Reagent setup
- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.

## Procedure
*Estimated timing: less than 1 minute*

1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.
2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.
3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.
4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).
   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.
5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.
6. Place empty Evotips, called Blanks, at S3 from A1 to A6.
7. Document the precise position of each placed Evotip.

## Expected Results
When the procedure is performed correctly, you should observe:
- Properly seated Evotip boxes in the rack system
- Visible Buffer A solution in boxes (1 cm depth)
- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip
- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.

## Figures

### Figure 1: Evosep positions
- Close-up of single Evotip box showing well positions (A1-H12)

### Figure 2: Evotip Quality Assessment
- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent
- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc

## References
1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf
2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf

**STEP 2: Documentation of observed actions with timestamps**

Table 1:
| Timestamp   | Visual/Audio Action                                                                                               |
|-------------|-------------------------------------------------------------------------------------------------------------------|
| [00:00:00]  | Video starts, showing the Evosep One system.                                                                      |
| [00:00:03]  | Researcher, wearing black gloves, picks up an Evotip box labeled "EVOTIPS SAMPLES HPLC".                            |
| [00:00:04]  | Researcher inspects the Evotip box, turning it to show the solution level (appears to be Buffer A, depth looks sufficient). |
| [00:00:09]  | Researcher places the Evotip box into the S1 position on the Evosep One rack.                                     |
| [00:00:12]  | Researcher picks up a second Evotip box, labeled "EVOTIP PURE".                                                   |
| [00:00:13]  | Researcher places the second Evotip box (for blanks, appears empty of solution) into the S3 position on the Evosep One rack. |
| [00:00:15]  | Researcher picks up a small, clear plastic container holding Evotips (for S1).                                      |
| [00:00:18]  | Researcher takes out two Evotips from the container.                                                              |
| [00:00:20]  | Researcher visually inspects the two Evotips. The SPE material discs appear pale and solvent is visible.          |
| [00:00:21]  | Researcher places one Evotip into position A1 of the S1 Evotip box.                                               |
| [00:00:22]  | Researcher places the second Evotip into position A2 of the S1 Evotip box.                                          |
| [00:00:23]  | Researcher takes out another two Evotips for S1.                                                                  |
| [00:00:24]  | Researcher visually inspects these two Evotips. Discs appear pale, solvent visible.                               |
| [00:00:25]  | Researcher places one Evotip into position A3 of the S1 Evotip box.                                               |
| [00:00:26]  | Researcher places the second Evotip into position A4 of the S1 Evotip box.                                          |
| [00:00:27]  | Researcher takes out another two Evotips for S1.                                                                  |
| [00:00:28]  | Researcher visually inspects these two Evotips. Discs appear pale, solvent visible.                               |
| [00:00:29]  | Researcher places one Evotip into position A5 of the S1 Evotip box.                                               |
| [00:00:30]  | Researcher places the second Evotip into position A6 of the S1 Evotip box.                                          |
| [00:00:31]  | Researcher puts down the container of S1 Evotips.                                                                 |
| [00:00:32]  | Researcher picks up a different, clear plastic container, presumably containing blank Evotips for S3.             |
| [00:00:34]  | Researcher takes out two blank Evotips.                                                                           |
| [00:00:35]  | Researcher visually inspects the two blank Evotips.                                                               |
| [00:00:36]  | Researcher places one blank Evotip into position A1 of the S3 Evotip box.                                         |
| [00:00:37]  | Researcher places the second blank Evotip into position A2 of the S3 Evotip box.                                    |
| [00:00:38]  | Researcher takes out another two blank Evotips.                                                                   |
| [00:00:39]  | Researcher visually inspects these two blank Evotips.                                                             |
| [00:00:40]  | Researcher places one blank Evotip into position A3 of the S3 Evotip box.                                         |
| [00:00:41]  | Researcher places the second blank Evotip into position A4 of the S3 Evotip box.                                    |
| [00:00:42]  | Researcher puts down the container of blank Evotips.                                                              |
| [00:00:45]  | Researcher points to the Evotips placed in S1 (positions A1-A6).                                                  |
| [00:00:48]  | Researcher points to the Evotips placed in S3 (positions A1-A4).                                                  |
| [00:00:52]  | Video ends.                                                                                                       |

**STEP 3: Systematic comparison**

Table 2:
| Step | Step Description | Timestamp in 'Video to evaluate' | Comparison Result | Notes |
|---|---|---|---|---|
| 1 | Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks. | [00:00:03] - [00:00:08] | Aligned | The researcher picked up the Evotip box for S1 ("EVOTIPS SAMPLES HPLC") and visually inspected the solution level by turning the box. The depth appeared to be at least 1 cm. No inspection of the S3 box's solution level was performed, which aligns with the protocol. |
| 2 | Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position. | [00:00:09] - [00:00:11] | Aligned | The Evotip box for S1 was placed in the S1 position on the Evosep rack. It appeared to be seated firmly. |
| 3 | Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position. | [00:00:12] - [00:00:14] | Aligned | An Evotip box labeled "EVOTIP PURE" (assumed for blanks and visibly empty of bulk solution) was placed in the S3 position. It appeared to be seated firmly. |
| 4 | Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2). **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis. | [00:00:20], [00:00:24], [00:00:28] (for S1 Evotips); [00:00:35], [00:00:39] (for S3 blank Evotips) | Aligned | The researcher inspected Evotips in pairs before placement. For S1 tips ([00:00:18]-[00:00:20], [00:00:23]-[00:00:24], [00:00:27]-[00:00:28]), the SPE material discs appeared pale, and solvent seemed visible. The same inspection was performed for the blank Evotips for S3 ([00:00:34]-[00:00:35], [00:00:38]-[00:00:39]). No Evotips were discarded, indicating all inspected tips met the quality criteria. |
| 5 | Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6. | [00:00:21] - [00:00:30] | Aligned | Six verified Evotips were placed sequentially into positions A1, A2, A3, A4, A5, and A6 of the Evotip box at S1. |
| 6 | Place empty Evotips, called Blanks, at S3 from A1 to A6. | [00:00:36] - [00:00:41] | ❌ **Error:** Only 4 of 6 specified blank Evotips were placed. | Blank Evotips were placed in the S3 box, but only into positions A1, A2, A3, and A4. Positions A5 and A6 were not filled as per the protocol. |
| 7 | Document the precise position of each placed Evotip. | No explicit documentation shown. Pointing observed at [00:00:45] - [00:00:51]. | ❌ **Omitted:** No written or electronic documentation observed. | The researcher pointed to the Evotips in S1 ([00:00:45]-[00:00:47]) and S3 ([00:00:48]-[00:00:51]). However, no act of writing down or electronically recording the positions was observed in the video. |

**STEP 4: Resulting lab notes**

# Placing Evotips in Evotip Boxes on the Evosep One System

## Aim
This protocol detailed the procedure for inspecting Evotips and placing them in Evotip boxes on the Evosep One liquid chromatography system.

## Materials

### Equipment
- Evotips
- Evotip Boxes
- Evosep One System

### Reagents
- Formic acid (FA)

### Reagent setup
- Buffer A: Consisted of 0.1% (vol/vol) FA.

## Procedure
*Timing: Approximately 48 seconds for core actions and confirmation.*

1.  The Evotip box intended for S1 was visually inspected to verify it was filled with Buffer A solution. The solution depth appeared to be a minimum of 1 cm ([00:00:03] - [00:00:08]). Verification of the Evotip box for S3 was not performed for solution level, as it was intended for blanks.
2.  The Evotip Box was placed at S1 within the rack system of the Evosep instrument. The box appeared to be firmly seated ([00:00:09] - [00:00:11]).
3.  An Evotip Box (labeled "EVOTIP PURE," empty of bulk solution) for Blank tips was placed at S3. The box appeared to be firmly seated ([00:00:12] - [00:00:14]).
4.  Each Evotip was inspected before placement to verify its condition. All inspected Evotips displayed a pale-colored SPE material disc with visible solvent above it. No Evotips were discarded ([00:00:20], [00:00:24], [00:00:28] for S1 tips; [00:00:35], [00:00:39] for S3 blank tips).
5.  The verified Evotips were placed into the prepared Evotip box at S1 from A1 to A6 ([00:00:21] - [00:00:30]).
6.  ❌ **Error:** Empty Evotips (Blanks) were placed at S3 only from A1 to A4 ([00:00:36] - [00:00:41]). The protocol specified placement from A1 to A6; thus, blanks for A5 and A6 were omitted.
7.  ❌ **Omitted:** The precise position of each placed Evotip was not documented visually (e.g., by writing or electronic entry). The researcher pointed to the placed tips ([00:00:45] - [00:00:51]), but this does not constitute documentation.

## Results
- Evotip boxes were observed to be properly seated in the S1 and S3 positions of the rack system.
- Visible Buffer A solution of approximately 1 cm depth was observed in the S1 Evotip box.
- All Evotips placed in S1 (A1-A6) showed pale-colored SPE material discs with a clear solvent meniscus.
- Blank Evotips placed in S3 (A1-A4) also appeared to be in good condition.
- Documentation of tip positions was not observed. The final configuration was Evotips at S1 in positions A1, A2, A3, A4, A5, A6, and blanks at S3 in positions A1, A2, A3, A4.

2025-05-08 10:38:54,958 - __main__ - INFO - Successfully extracted and parsed JSON.


Time to process and evaluate lab notes: 91.26 seconds


## Lab notes Quality
| Criterion | Rating    | Explanation                                                                                                                                                                                                                                                                                                                                                        |
|-----------|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Structure | Good      | The AI successfully kept the essential sections: Aim, Materials, Procedure, and Results. The "Materials" section was slightly restructured by adding a "Reagents" subsection, which is an acceptable variation. However, the "Aim" section in the AI-generated notes ("This protocol detailed the procedure for...") described the general purpose of the input protocol document rather than stating the specific experimental objective of the lab session (e.g., specific samples or target positions), which was more clearly defined in the benchmark's Aim. |
| Tense     | Excellent | The AI consistently used the past tense throughout the Procedure and Results sections to accurately describe what was observed and performed (e.g., "was visually inspected," "were placed," "were observed"). This aligns with the requirement to describe what actually happened.                                                                                  |
| Language  | Excellent | The AI effectively removed instructional language (e.g., "Visually inspect...") present in the original protocol (as shown in its STEP 1) and replaced it with observational language in the generated lab notes (e.g., "The Evotip box... was visually inspected..."). The descriptions are based on observed actions.                                        |
| Numbering | Excellent | The AI maintained the step numbering (1-7) from the original protocol in its Procedure section. It correctly identified and noted deviations from the protocol (an error in step 6 and an omission in step 7) while preserving the original numbering structure.                                                                                                    |
| Timing    | Excellent | The AI included specific timing information. It provided an overall duration ("Approximately 48 seconds for core actions and confirmation") and also embedded precise timestamps for individual actions within the procedure steps (e.g., "[00:00:03] - [00:00:08]"). This fulfills the requirement for exact actual timing, surpassing the benchmark's estimated timing. |

...

Time to process and evaluate lab notes: 136.85 seconds


/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/ipykernel_64238/2452223971.py:519: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_errors = df_error_benchmark.merge(df_error_ai, on="Step", how="outer")


## Lab notes Quality
| Criterion | Rating    | Explanation                                                                                                                                                                                                                                                              |
|-----------|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Structure | Excellent | The AI-generated lab notes correctly used only the relevant sections: Aim, Materials, Procedure, and Results, perfectly matching the structure of the benchmark lab notes.                                                                                             |
| Tense     | Excellent | The AI consistently used past tense throughout the lab notes to describe what actually happened during the experiment (e.g., "The researcher began...", "parameters were configured...", "Evotips commenced spinning..."), adhering to the requirement.                  |
| Language  | Poor      | While the AI successfully removed instructional language and used an observational style, a key observation within its lab notes directly contradicts the benchmark ground truth. The AI states the Evotips were spun at 800g, but the benchmark specifies they were spun at 700g. This discrepancy in a critical experimental parameter constitutes a significant issue with the recorded "observation." |
| Numbering | Excellent | The AI maintained the step numbering (1, 2, 3, 4) from the original protocol for corresponding actions. It appropriately incorporated newly observed actions or sub-steps with additional clear numbering (e.g., 0.1, 3.1, 3.2, 3.3) without disrupting the core protocol's numerical flow. |
| Timing    | Excellent | The AI included exact actual timing. It correctly reported the set centrifuge run time ("1 minute") and enhanced the notes by providing specific timestamps from the video for various actions (e.g., "00:00:25 - 00:00:29" for parameter configuration). This is an improvement over estimated timings. |

,Step,Benchmark,Class,Skill,AI Response,AI Class,Identification,Classification
0,0.1,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
1,0.2,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
2,1.0,Error,Error,ProteomicsKnowledge,Error,Deviation & Error,Error (Correctly Identified),incorrect
3,2.0,Error,Omitted,GeneralKnowledge,Error,Error,Error (Correctly Identified),incorrect
4,3.0,No Error,N/A,N/A,No Error,N/A,No Error (Correctly Identified),N/A
5,3.1,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
6,3.2,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
7,3.3,NaN,NaN,NaN,Error,Addition,Addition by model,N/A
8,4.0,Error,Error,Spatial,No Error,N/A,False Negative,N/A


Waiting 10 seconds before next item...
All processing complete. Final results saved.
